# Ant-Colony-Optimization Problem

In [1]:
import pandas as pd
import numpy as np

Importing .txt files and saving them as arrays.

In [12]:
with open('01_tsp.txt', 'r') as t1:
    array1 = []
    for line in t1.readlines():
        y = [int(value) for value in line.split()]
        array1.append(y)

    t1.close()
    array1 = np.array(array1)
    print(array1)

print(array1.shape)

print(type(array1))


[[  0 100 229 ... 379 183 178]
 [100   0 329 ... 479 231 278]
 [229 329   0 ... 150 290  87]
 ...
 [379 479 150 ...   0 248 201]
 [183 231 290 ... 248   0 203]
 [178 278  87 ... 201 203   0]]
(150, 150)
<class 'numpy.ndarray'>


In [43]:
def evaporate(pheromone_matrix):
    pheromone_matrix= pheromone_matrix * 0.1

    return pheromone_matrix


## Intensification


solution_matrix will be a list of lists which hold the numbers of the cities in the order that the each of the ants visited them


In [45]:
array2=[[1, 1, 1],[2, 2, 2],[3, 3, 3]]
array3=[[1, 2, 2],[2, 2, 1]]
def intensification(pheromone_matrix,solution_matrix):
    
    for i in solution_matrix:
    
        x=0
        for j in i:
          
            pheromone_matrix[x][j]= pheromone_matrix[x][j]*1.5
            x=j
    return pheromone_matrix
            
print(intensification(array2,array3))

[[1, 1.5, 1.5], [2, 2, 3.0], [3, 4.5, 6.75]]


[[ 0.  10.  22.9 ... 37.9 18.3 17.8]
 [10.   0.  32.9 ... 47.9 23.1 27.8]
 [22.9 32.9  0.  ... 15.  29.   8.7]
 ...
 [37.9 47.9 15.  ...  0.  24.8 20.1]
 [18.3 23.1 29.  ... 24.8  0.  20.3]
 [17.8 27.8  8.7 ... 20.1 20.3  0. ]]
